In [1]:
from google.colab import files
from IPython.display import clear_output

In [2]:
!pip3 install ufal.udpipe
clear_output()

In [3]:
from typing import List

from tqdm import tqdm
from ufal.udpipe import Model, Pipeline

In [4]:
!wget https://rusvectores.org/static/models/udpipe_syntagrus.model

--2022-03-21 12:27:49--  https://rusvectores.org/static/models/udpipe_syntagrus.model
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40616122 (39M)
Saving to: ‘udpipe_syntagrus.model’

udpipe_syntagrus.mo 100%[===================>]  38.73M   157MB/s    in 0.2s    

2022-03-21 12:27:50 (157 MB/s) - ‘udpipe_syntagrus.model’ saved [40616122/40616122]



In [5]:
model_udpipe = Model.load("udpipe_syntagrus.model")
pipeline = Pipeline(
    model_udpipe, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
    )

In [6]:
!wget https://raw.githubusercontent.com/vyhuholl/russian_detoxification/master/data/test.txt

--2022-03-21 12:29:45--  https://raw.githubusercontent.com/vyhuholl/russian_detoxification/master/data/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1941971 (1.9M) [text/plain]
Saving to: ‘test.txt’

test.txt            100%[===================>]   1.85M  --.-KB/s    in 0.03s   

2022-03-21 12:29:46 (71.0 MB/s) - ‘test.txt’ saved [1941971/1941971]



In [7]:
!wget https://raw.githubusercontent.com/vyhuholl/russian_detoxification/master/data/toxic_vocab.txt

--2022-03-21 12:29:48--  https://raw.githubusercontent.com/vyhuholl/russian_detoxification/master/data/toxic_vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7618338 (7.3M) [text/plain]
Saving to: ‘toxic_vocab.txt’

toxic_vocab.txt     100%[===================>]   7.26M  --.-KB/s    in 0.05s   

2022-03-21 12:29:49 (139 MB/s) - ‘toxic_vocab.txt’ saved [7618338/7618338]



In [8]:
def tokenize(
    text: str, pipeline: Pipeline, tags: bool = False, lemmas: bool = False
) -> List[str]:
    """
    Tokenizes a text with the UDPipe pipeline.

    Parameters:
        inputs_path: str
        preds_path: str
        results_file: str
    Returns:
        None
    """
    processed = pipeline.process(text)
    content = [
        line for line in processed.split("\n") if not line.startswith("#")
    ]
    tagged = [w.split("\t") for w in content if w]
    tokens = []

    for token in tagged:
        if token[3] == "PUNCT":
            continue
        token_res = ""
        if lemmas:
            token_res = token[2]
        else:
            token_res = token[1]
        if tags:
            token_res += "_" + token[3]
        tokens.append(token_res)

    return tokens

In [12]:
with open("test.txt") as inputs_file:
    texts = inputs_file.readlines()

with open("toxic_vocab.txt") as vocab_file:
    vocab = [line.strip() for line in vocab_file.readlines()]

In [13]:
results = []

for text in tqdm(texts, desc="Deleting toxic words..."):
    words = tokenize(text, pipeline, lemmas=False)
    lemmas = tokenize(text, pipeline, lemmas=True)
    clean_text = " ".join(
        [word for word, lemma in zip(words, lemmas) if lemma not in vocab]
        )
    results.append(clean_text)

Deleting toxic words...: 100%|██████████| 12358/12358 [17:29<00:00, 11.77it/s]


In [14]:
with open("preds_delete.txt", "w") as results_file:
    for text in results:
        results_file.write(text + "\n")

In [15]:
!head test.txt

вот это точьно пидорасы
скальпы местного населения за деньги не принимали, не резали руки детям за отказ их родителей работать и невыполнение плана, Какой потрясающе удобный подход. А еще не набивали соломой трупы врагов и не насиловали предметы домашнего обихода туземцев. Что не добавили и эти два пункта? Я привел тебе факты многочисленных убийств аборигенов ради захвата их земель или подчинения их народов, а ты ловко выкручиваешься тем, что вырезали-то не всех поголовно, а всего лишь непокорные аулы, концлагери строили не для конкретных народов, а всех подряд, да и резервации не строили - ведь просто выкинуть из поезда в сухую степь Средней Азии полмиллиона насильно депортированных из родных краев вайнахов и сколько-то там других народов было гораздо проще, чем что-то для них еще и строить. Войны были есть и будут Это была экспансия, а не война. Разницу чуешь, или мне снова разжевать? Так что нехер пытаться обелить то, что было. Не сразу заметил, что ты еще и автор этого поста, где ж

In [16]:
!head preds_delete.txt

вот это точьно
скальпы местного населения за деньги не принимали не резали руки детям за отказ их родителей работать и невыполнение плана Какой потрясающе удобный подход А еще не набивали соломой трупы врагов и не насиловали предметы домашнего обихода туземцев Что не добавили и эти два пункта Я привел тебе факты многочисленных убийств аборигенов ради захвата их земель или подчинения их народов а ты ловко выкручиваешься тем что вырезали-то не всех поголовно а всего лишь непокорные аулы концлагери строили не для конкретных народов а всех подряд да и резервации не строили ведь просто выкинуть из поезда в сухую степь Средней Азии полмиллиона насильно депортированных из родных краев вайнахов и сколько-то там других народов было гораздо проще чем что-то для них еще и строить Войны были есть и будут Это была экспансия а не война Разницу чуешь или мне снова разжевать Так что нехер пытаться обелить то что было Не сразу заметил что ты еще и автор этого поста где жалуешься на мертвое образование 

In [17]:
files.download("preds_delete.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>